7. In Sections 5.1.2 and 5.1.3, we saw that the cross_validate() function can be used in order to compute the LOOCV test error estimate. Alternatively, one could compute those quantities using just sm.GLM() and the predict() method of the fitted model within a for loop. You will now take this approach in order to compute the LOOCV error for a simple logistic regression model on the Weekly data set. Recall that in the context of classification problems, the LOOCV error is given in (5.4).

In [12]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler


In [32]:
weekly = pd.read_csv('~/Independent Study /Weekly.csv')
weekly 

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,Down
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,Down
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,Up
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,Up
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,Up
...,...,...,...,...,...,...,...,...,...
1084,2010,-0.861,0.043,-2.173,3.599,0.015,3.205160,2.969,Up
1085,2010,2.969,-0.861,0.043,-2.173,3.599,4.242568,1.281,Up
1086,2010,1.281,2.969,-0.861,0.043,-2.173,4.835082,0.283,Up
1087,2010,0.283,1.281,2.969,-0.861,0.043,4.454044,1.034,Up


In [33]:
weekly.columns

Index(['Year', 'Lag1', 'Lag2', 'Lag3', 'Lag4', 'Lag5', 'Volume', 'Today',
       'Direction'],
      dtype='object')

(a) Fit a logistic regression model that predicts Direction using Lag1 and Lag2.

In [34]:
# Convert Direction to a binary variable
weekly['Direction_Binary'] = (weekly['Direction'] == 'Up').astype(int)

# Set up the predictor variables (Lag1 and Lag2) and response variable (Direction_Binary)
X = weekly[['Lag1', 'Lag2']]
X = sm.add_constant(X)  # Adding a constant for the intercept
y = weekly['Direction_Binary']

# Fit the logistic regression model
model = sm.GLM(y, X, family=sm.families.Binomial())
result = model.fit()
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:       Direction_Binary   No. Observations:                 1089
Model:                            GLM   Df Residuals:                     1086
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -744.11
Date:                Fri, 06 Oct 2023   Deviance:                       1488.2
Time:                        09:02:27   Pearson chi2:                 1.09e+03
No. Iterations:                     4   Pseudo R-squ. (CS):           0.007303
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2212      0.061      3.599      0.0

(b) Fit a logistic regression model that predicts Direction using Lag1 and Lag2 using all but the first observation.

In [36]:
# Drop the first observation
weekly_minus_first = weekly.iloc[1:, :]

# Set up the predictor variables (Lag1 and Lag2) and response variable (Direction_Binary) for the modified dataset
X_minus_first = weekly_minus_first[['Lag1', 'Lag2']]
X_minus_first = sm.add_constant(X_minus_first)  # Adding a constant for the intercept
y_minus_first = weekly_minus_first['Direction_Binary']

# Fit the logistic regression model using all but the first observation
model_minus_first = sm.GLM(y_minus_first, X_minus_first, family=sm.families.Binomial())
result_minus_first = model_minus_first.fit()
print(result_minus_first.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:       Direction_Binary   No. Observations:                 1088
Model:                            GLM   Df Residuals:                     1085
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -743.26
Date:                Fri, 06 Oct 2023   Deviance:                       1486.5
Time:                        09:04:00   Pearson chi2:                 1.09e+03
No. Iterations:                     4   Pseudo R-squ. (CS):           0.007373
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2232      0.061      3.630      0.0

(c) Use the model from (b) to predict the direction of the first obser- vation. You can do this by predicting that the first observation will go up if P (Direction = "Up"|Lag1, Lag2) > 0.5. Was this observation correctly classified?

In [40]:
print(X_first)


    Lag1   Lag2
0  0.816  1.572


In [41]:
# Given code to fit the model without the first observation:
weekly_minus_first = weekly.iloc[1:, :]

# Set up the predictor variables (Lag1 and Lag2) and response variable (Direction_Binary) for the modified dataset
X_minus_first = weekly_minus_first[['Lag1', 'Lag2']]
X_minus_first = sm.add_constant(X_minus_first)  # Adding a constant for the intercept
y_minus_first = weekly_minus_first['Direction_Binary']

# Fit the logistic regression model using all but the first observation
model_minus_first = sm.GLM(y_minus_first, X_minus_first, family=sm.families.Binomial())
result_minus_first = model_minus_first.fit()
print(result_minus_first.summary())

# Extract features for the first observation
X_first = weekly.iloc[0:1][['Lag1', 'Lag2']]
X_first = sm.add_constant(X_first)  # Ensure the intercept is added

# Predict the direction for the first observation
predicted_probability = result_minus_first.predict(X_first)

# Determine the predicted direction: 1 if P(Direction = "Up"|Lag1, Lag2) > 0.5, otherwise 0
predicted_direction = (predicted_probability > 0.5).astype(int).values[0]

# Check if the prediction was correct
actual_direction = weekly.iloc[0]['Direction_Binary']
correctly_classified = predicted_direction == actual_direction

print(f"Predicted direction: {'Up' if predicted_direction == 1 else 'Down'}")
print(f"Actual direction: {weekly.iloc[0]['Direction']}")
print(f"Correctly classified: {correctly_classified}")




                 Generalized Linear Model Regression Results                  
Dep. Variable:       Direction_Binary   No. Observations:                 1088
Model:                            GLM   Df Residuals:                     1085
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -743.26
Date:                Fri, 06 Oct 2023   Deviance:                       1486.5
Time:                        09:09:51   Pearson chi2:                 1.09e+03
No. Iterations:                     4   Pseudo R-squ. (CS):           0.007373
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2232      0.061      3.630      0.0

ValueError: shapes (1,2) and (3,) not aligned: 2 (dim 1) != 3 (dim 0)

In [42]:
# Extract features for the first observation
X_first = weekly.iloc[0:1][['Lag1', 'Lag2']]

# Manually add the intercept (constant) column
X_first.insert(0, 'const', 1.0)

# Predict the direction for the first observation
predicted_probability = result_minus_first.predict(X_first)

# Determine the predicted direction: 1 if P(Direction = "Up"|Lag1, Lag2) > 0.5, otherwise 0
predicted_direction = (predicted_probability > 0.5).astype(int).values[0]

# Check if the prediction was correct
actual_direction = weekly.iloc[0]['Direction_Binary']
correctly_classified = predicted_direction == actual_direction

print(f"Predicted direction: {'Up' if predicted_direction == 1 else 'Down'}")
print(f"Actual direction: {weekly.iloc[0]['Direction']}")
print(f"Correctly classified: {correctly_classified}")


Predicted direction: Up
Actual direction: Down
Correctly classified: False
